# Visualize energy landscape

1. Pull a model from W&B
2. Convert the model to ONNX (scripts/pt_to_onnx.py)
3. Run this notebook
4. Run visualization (scripts/visualize_energy.py)

In [2]:

from pathlib import Path
import sys

import torch
import numpy as np
from torchvision import transforms
from tqdm.auto import tqdm
import onnxruntime as ort

sys.path.append('../')

from dataloading.nvidia import NvidiaCropWide, Normalize, NvidiaDataset

In [3]:
def get_energy(model, x, y):
    inputs = {'x': x, 'y': y}
    return model.run(None, inputs)[0].squeeze()

def get_dataset_energy(model, dataloader, n_samples=256, bound=1.5):
    all_energies = []
    progress_bar = tqdm(total=len(dataloader), smoothing=0)
    batch_size = dataloader.batch_size

    samples = np.linspace(-bound, bound, num=n_samples, dtype=np.float32).reshape(1, -1).repeat(batch_size, axis=0)
    samples = np.expand_dims(samples, axis=-1)

    for i, (input, _, __) in enumerate(dataloader):

        if input['image'].shape[0] != batch_size:
            print(f'Dropping batch with {input["image"].shape[0]} samples')
            continue

        inputs = input['image'].cpu().numpy()
        energy = get_energy(model, inputs, samples)

        all_energies.extend(energy)
        progress_bar.update(1)


    result = np.array(all_energies)
    return result, samples[0]

In [4]:
# model = ort.InferenceSession("../_models/ebm-regularized-v1-energy.onnx", providers=['CUDAExecutionProvider'])
model = ort.InferenceSession("../_models/ebm-regularized-v1-energy.onnx")

RuntimeError: /onnxruntime_src/onnxruntime/core/platform/posix/env.cc:183 onnxruntime::{anonymous}::PosixThread::PosixThread(const char*, int, unsigned int (*)(int, Eigen::ThreadPoolInterface*), Eigen::ThreadPoolInterface*, const onnxruntime::ThreadOptions&) pthread_setaffinity_np failed, error code: 11 error msg: Resource temporarily unavailable


In [7]:
root_path = Path("/data/Bolt/drives-nikita-thesis/")
valid_paths = [root_path / "2022-07-11-15-22-50_test_vahi_ibc_dfo_drop_frames"]

_, samples = model.get_inputs()
batch_size, n_samples, _ = samples.shape

print('batch_size:', batch_size, 'n_samples:', n_samples)

# TODO: try CV2 resize as done during inference
tr = transforms.Compose([NvidiaCropWide(), Normalize()])
inference_set = NvidiaDataset(valid_paths, transform=tr)

inference_loader = torch.utils.data.DataLoader(inference_set, batch_size=batch_size, shuffle=False,
                                         num_workers=1, pin_memory=False, persistent_workers=False)

ValueError: not enough values to unpack (expected 2, got 1)

In [6]:
energy, samples = get_dataset_energy(model, inference_loader, n_samples)

  0%|          | 0/594 [00:00<?, ?it/s]

Dropping batch with 16 samples


In [7]:
np.save('energy_valley.npy', energy)
np.save('samples.npy', samples)

In [8]:
[energy.shape, samples.shape]

[(18976, 1024), (1024, 1)]